In [1]:
#coding=utf-8
import jieba
import random
from collections import defaultdict

In [2]:

fail = [True, None]
nonsense = "Sorry, I don't know what you're  talking about."

def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])


def pat_match(pattern, saying):
    if is_variable(pattern[0]): return True
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

        
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

def subsitite(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])



def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    
    return (seg_pat, saying), len(saying)

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])    

def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}



In [3]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*x AI ?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

saying= [
    'Hello',
    "I think I want a toy",
    "I was a good boy",
    "I don't feel so good",
    "I feel sad",
    "我和你一样高兴",
    "我是钢铁侠",
    "我没出门因为今天下雨",
    "我觉得我不能现在就放弃",
    "我喜欢瑞克",
    "你觉得他们是坏人吗"
]

In [4]:
def preprocess(saying, is_pat=True):
    if is_pat:
        saying_array = jieba.lcut(saying)
        res = []
        i,m = 0,len(saying_array)
        while i < m:
            
            token = saying_array[i]
            if token.isalpha():
                res.append(token)
                i+=1
            elif token == '?' and i < m-1:
                j = i+2 if  saying_array[i+1] == '*' else i+1
                var = ''.join(saying_array[i:j+1])
                res.append(var)
                i = j+1
            else:
                i+=1
        return res                                
    else:
        return [s for s in jieba.lcut(saying) if s.isalpha()]


def is_failed_match(matched_pattern, saying_array):
    if  matched_pattern == fail:
        return True
    # if all variables add up equal to  saying_array, the match may be wrong
    all_vars = [val for vals in matched_pattern for val in vals[1]]
    if all_vars == saying_array:
        return True
    return False

def get_response(saying, response_rules):
    if not isinstance(saying,str) or not isinstance(response_rules, dict):
        return nonsense
    
    for pattern, possible_reponse in response_rules.items():
        saying_array = preprocess(saying,False)
        pattern_array = preprocess(pattern)
        matched_pattern = pat_match_with_seg(pattern_array,saying_array)
        # print(matched_pattern)
        if not is_failed_match(matched_pattern, saying_array):
            res = random.choice(possible_reponse)
            return ' '.join(subsitite(res.split(),pat_to_dict(matched_pattern)))
    return nonsense


def test():
    for s in saying:
        print("{} ==> {}".format(s,get_response(s,rule_responses)))


test()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\SHADYR~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.889 seconds.
Prefix dict has been built succesfully.


Hello ==> Sorry, I don't know what you're  talking about.
I think I want a toy ==> what would it mean if you got a toy
I was a good boy ==> Were you really
I don't feel so good ==> Sorry, I don't know what you're  talking about.
I feel sad ==> What other feelings do you have?
我和你一样高兴 ==> 你以前梦见过?y吗
我是钢铁侠 ==> 真的吗? --- ?y
我没出门因为今天下雨 ==> 你在醒着的时候，以前想象过?y吗？
我觉得我不能现在就放弃 ==> 真的吗? --- ?y
我喜欢瑞克 ==> ?y有什么好的呢？
你觉得他们是坏人吗 ==> 你觉得他们可能不是?y？


## map search

In [5]:
BEIJING, CHANGCHUN, MULUMUQI, WUHAN, GUNAGHZOU, SHENZHEN, BANGKOK, SHANGHAI, NEWYORK = """
BEIJING CHANGCHUN MULUMUQI WUHAN GUANGZHOU SHENZHEN BANGKOK SHANGHAI NEWYORK
""".split()
connection = {
    CHANGCHUN: [BEIJING],
    MULUMUQI: [BEIJING], 
    BEIJING: [MULUMUQI, CHANGCHUN, WUHAN, SHENZHEN, NEWYORK],
    NEWYORK: [BEIJING, SHANGHAI],
    SHANGHAI: [NEWYORK, WUHAN],
    WUHAN: [SHANGHAI, BEIJING, GUNAGHZOU],
    GUNAGHZOU: [WUHAN, BANGKOK],
    SHENZHEN: [WUHAN, BANGKOK],
    BANGKOK: [SHENZHEN, GUNAGHZOU]
}

In [6]:
def navigator_bfs(start,end,graph):
    if start == end:
        return [start]
    
    possible_paths = [[start]]
    seen = set()
    
    while possible_paths:
        path = possible_paths.pop(0)
        frontier = path[-1]
        if frontier not in seen:
            
            seen.add(frontier)
            successors = graph[frontier]
            for next_station in successors:
                
                path.append(next_station)
                if next_station == end:
                    return path
                possible_paths.append([p for p in path])
                path.pop()
        
    return None


print ('=>'.join(navigator_bfs(BEIJING,SHANGHAI,connection)))
print ('=>'.join(navigator_bfs(BEIJING,WUHAN,connection)))        
print('=>'.join(navigator_bfs(BEIJING,NEWYORK,connection)))
        
    

BEIJING=>WUHAN=>SHANGHAI
BEIJING=>WUHAN
BEIJING=>NEWYORK


In [70]:
def navigator_dfs(start,end,graph):
    path = [start]
    seen =set({start})
    def helper(start,end,graph,path,seen):
        if start == end:
            return path   
        successors = graph[start]
        for next_station in successors:
            if next_station not in seen:
                seen.add(next_station)
                path.append(next_station)
                if helper(next_station,end,graph,path,seen):
                    return path
                path.pop()
    helper(start,end,graph,path,seen)
    return path
    

# navigator_dfs(BEIJING,SHANGHAI,connection)
navigator_dfs(WUHAN,BEIJING,connection)

['WUHAN', 'SHANGHAI', 'NEWYORK', 'BEIJING']

## gramer

In [49]:
grammar = """
sentence => noun_phrase verb_phrase 
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*
verb_phrase => verb noun_phrase
Article =>  一个 | 这个
noun =>   女人 |  篮球 | 桌子 | 小猫
verb => 看着   |  坐在 |  听着 | 看见
Adj =>   蓝色的 |  好看的 | 小小的
"""


In [50]:
def parse_grammar(grammar_str, sep='=>'):
    grammar = {}
    for line in grammar_str.split('\n'): 
        line = line.strip()
        if not line: continue
        
        target, rules = line.split(sep)
        
        grammar[target.strip()] = [r.split() for r in rules.split('|')]
    
    return grammar

In [57]:
g = parse_grammar(grammar)
g

{'Adj': [['蓝色的'], ['好看的'], ['小小的']],
 'Adj*': [['null'], ['Adj', 'Adj*']],
 'Article': [['一个'], ['这个']],
 'noun': [['女人'], ['篮球'], ['桌子'], ['小猫']],
 'noun_phrase': [['Article', 'Adj*', 'noun']],
 'sentence': [['noun_phrase', 'verb_phrase']],
 'verb': [['看着'], ['坐在'], ['听着'], ['看见']],
 'verb_phrase': [['verb', 'noun_phrase']]}

In [60]:
def generate(grammar,target='sentence'):
    if target not in grammar:return target
    rule = random.choice(grammar[target])
    print(rule)
    return ''.join(generate(grammar,r) for r in rule if r != 'null')

generate(g)

['noun_phrase', 'verb_phrase']
['Article', 'Adj*', 'noun']
['这个']
['Adj', 'Adj*']
['小小的']
['null']
['小猫']
['verb', 'noun_phrase']
['看着']
['Article', 'Adj*', 'noun']
['一个']
['null']
['小猫']


'这个小小的小猫看着一个小猫'